In [2]:
import pandas as pd
import numpy as np

In [ ]:
def get_title_clean(df, col):
    book_title = df[col]
    #book_title = amazon_book_title.dropna()
    book_title = book_title.str.lower()
    book_title = book_title.str.replace(r'[\]\(\)\\\'\":;]', '')
    book_title = book_title.str.replace(r'\[', ' ')
    return book_title

## Book genre addition

In [ ]:
df_book_meta = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/book-meta-valid.csv')
df_book_meta.head()

In [ ]:
df_book_genres = pd.read_csv('/home/ignacio/Datasets/Graph analysis/bx-valid-book.csv')
df_book_genres.head()

In [ ]:
df_book_meta['title clean'] = get_title_clean(df_book_meta, 'title')
df_book_meta.head()

In [ ]:
df_book_meta = df_book_meta[['title', 'asin', 'description', 'title clean']]
df_book_meta.head()

In [ ]:
df_book_meta.isnull().any()

In [ ]:
df_book_meta['description'] = df_book_meta['description'].fillna('no description')

In [ ]:
df_book_genres['title clean'] = get_title_clean(df_book_genres, 'Book-Title')
df_book_genres.head()

In [ ]:
df_book_genres.isnull().any()

In [ ]:
df_book_genres = df_book_genres.fillna('no description')

In [ ]:
df_book_meta['shelves'] = df_book_meta['title clean'].apply(
    lambda x : df_book_genres[df_book_genres['title clean'].str.contains(x)]['common-shelves'].drop_duplicates().values)
df_book_meta.head()

In [ ]:
df_book_meta['shelves'] = df_book_meta['shelves'].apply(
    lambda shels : '|'.join([x for x in shels if x != 'no description']))
df_book_meta.head()

In [ ]:
df_book_meta['author'] = df_book_meta['title clean'].apply(
    lambda x : df_book_genres[df_book_genres['title clean'].str.contains(x)]['Book-Author'].drop_duplicates().values)

In [ ]:
df_book_meta['author'] = df_book_meta['author'].apply(lambda x : x[0] if len(x)!=0 else 'no author')
df_book_meta.head()

In [ ]:
df_book_meta['ISBN'] = df_book_meta['title clean'].apply(
    lambda x : df_book_genres[df_book_genres['title clean'].str.contains(x)]['ISBN'].drop_duplicates().values)
df_book_valid.head()

In [ ]:
df_book_meta.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/book_meta_valid_shelves.csv', index=False)

### Book average rating add (from GoodReads)

In [1]:
from goodreads import client, shelf
import oauth2 as oauth
import urllib


gc = client.GoodreadsClient('63pG3gIBwuO4vq83YxHpw', 'P4t2Nvwesy7h4DG9hTKdBbdQHlo76Uz7Nxd6iFJZU')

In [3]:
df_book_meta = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/book_meta_valid_shelves.csv')
df_book_meta.head()

,ISBN,Book-Title,Book-Author,common-shelves,description,asin
0,0399135782,The Kitchen God's Wife,Amy Tan,romance|war|drama,Tan can relax. If The Joy Luck Club was an ast...,0143038109
1,0446310786,To Kill a Mockingbird,Harper Lee,crime|mystery|drama,"""When he was nearly thirteen, my brother Jem g...",0060194995
2,055321215X,Pride and Prejudice,Jane Austen,historical-romance|drama,no description,0030514878
3,0553582747,From the Corner of His Eye,Dean Koontz,fantasy|urban-fantasy|horror-suspense|suspense...,Horrormeister Koontz looks heavenward for insp...,0553502697
4,0380715899,A Soldier of the Great War,Mark Helprin,the-great-war|adventure|war-memoirs|world-war-...,"With energetic, often lyrical prose capable of...",0151836000


In [7]:
def get_average_rating( isbn ):
    try:
        book = gc.book(isbn=isbn)
        result = book.average_rating
    except Exception:
        print('INVALID ISBN ' + isbn)
        result = 0
    return result

In [9]:
import math

frame = 100
max_complete_frames = math.floor(len(df_book_meta)/frame)
max_complete_frames

62

In [10]:
for i in np.arange(0, max_complete_frames):
    df_aux = df_book_meta.loc[i*frame : (i+1)*frame]
    df_aux['average rating'] = df_aux['ISBN'].apply(get_average_rating)
    df_aux.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/Book_rating_'+ str(i) +'.csv', index=False)

df_book_meta.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


INVALID ISBN B000069F44
INVALID ISBN 0378023624
INVALID ISBN 0340213361
INVALID ISBN 840100926X
INVALID ISBN 0553408909
INVALID ISBN 020168859x
INVALID ISBN 1570821739
INVALID ISBN 3822895849
INVALID ISBN 8476729332
INVALID ISBN B00013AFZQ
INVALID ISBN 0739436376
INVALID ISBN 1404354077
INVALID ISBN 8425400538
INVALID ISBN 140432741x
INVALID ISBN 0394178092
INVALID ISBN 0449210340
INVALID ISBN 0449210359
INVALID ISBN 0091832314
INVALID ISBN B00005TZWI
INVALID ISBN 0718217217
INVALID ISBN 087220460x


,ISBN,Book-Title,Book-Author,common-shelves,description,asin
0,0399135782,The Kitchen God's Wife,Amy Tan,romance|war|drama,Tan can relax. If The Joy Luck Club was an ast...,0143038109
1,0446310786,To Kill a Mockingbird,Harper Lee,crime|mystery|drama,"""When he was nearly thirteen, my brother Jem g...",0060194995
2,055321215X,Pride and Prejudice,Jane Austen,historical-romance|drama,no description,0030514878
3,0553582747,From the Corner of His Eye,Dean Koontz,fantasy|urban-fantasy|horror-suspense|suspense...,Horrormeister Koontz looks heavenward for insp...,0553502697
4,0380715899,A Soldier of the Great War,Mark Helprin,the-great-war|adventure|war-memoirs|world-war-...,"With energetic, often lyrical prose capable of...",0151836000


In [12]:
if len(df_book_meta)%frame != 0 :
    df_aux = df_book_meta.loc[max_complete_frames*frame:]
    df_aux['average rating'] = df_aux['ISBN'].apply(get_average_rating)
    df_aux.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/Book_rating_'+ str(max_complete_frames) +'.csv', index=False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
df_book_meta = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/Book_rating_0.csv')
df_book_meta.head()

,ISBN,Book-Title,Book-Author,common-shelves,description,asin,average rating
0,0399135782,The Kitchen God's Wife,Amy Tan,romance|war|drama,Tan can relax. If The Joy Luck Club was an ast...,0143038109,3.99
1,0446310786,To Kill a Mockingbird,Harper Lee,crime|mystery|drama,"""When he was nearly thirteen, my brother Jem g...",0060194995,4.26
2,055321215X,Pride and Prejudice,Jane Austen,historical-romance|drama,no description,0030514878,4.25
3,0553582747,From the Corner of His Eye,Dean Koontz,fantasy|urban-fantasy|horror-suspense|suspense...,Horrormeister Koontz looks heavenward for insp...,0553502697,4.01
4,0380715899,A Soldier of the Great War,Mark Helprin,the-great-war|adventure|war-memoirs|world-war-...,"With energetic, often lyrical prose capable of...",0151836000,4.35


In [14]:
for i in np.arange(1, max_complete_frames + 1):
    df_aux = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/Book_rating_'+str(i)+'.csv')
    df_book_meta = pd.concat([df_book_meta, df_aux])
len(df_book_meta)

6292

In [15]:
df_book_meta.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/book_meta_valid_shelves_rating.csv', index=False)

## Movie genre addition

In [ ]:
df_movie_meta = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_meta_valid.csv')
df_movie_meta.head()

In [ ]:
df_movie_genre = pd.read_csv('/home/ignacio/Datasets/Graph analysis/ml-valid-movies.csv')
df_movie_genre.head()

In [ ]:
df_movie_meta = df_movie_meta[['asin','title', 'description']]
df_movie_meta.head()

In [ ]:
df_movie_meta = df_movie_meta.fillna('no description')

In [ ]:
df_movie_meta['title'] = df_movie_meta['title'].str.replace(r' ?\[VHS\] ?', '')
df_movie_meta['title'] = df_movie_meta['title'].str.replace(r' ?DVD ?', '')
df_movie_meta.head()

In [ ]:
df_movie_genre['title clean'] = get_title_clean(df_movie_genre, 'title')
df_movie_genre.head()

In [ ]:
df_movie_meta['title clean'] = get_title_clean(df_movie_meta, 'title')
df_movie_meta.head()

In [ ]:
df_movie_meta['genres'] = ''
df_movie_meta['director'] = ''
df_movie_meta.head()

In [ ]:
import re

def get_asins(row, df_movie_meta):
    pattern = '^' + row['title clean'] +'$'
    regex = re.compile(pattern)
    asins = df_movie_meta[df_movie_meta['title clean'].str.contains(pattern)]['asin'].drop_duplicates()
    df_movie_meta['genres'] = df_movie_meta.apply(lambda x : x['genres'] + '|' +row['genres'] if x['asin'] in asins.values else x['genres'], axis=1)
    df_movie_meta['director'] = df_movie_meta.apply(lambda x: x['director'] + ', ' +row['director'] if x['asin'] in asins.values else x['director'], axis=1)
    return True

In [ ]:
frame = 100
max_frames = int(np.floor(len(df_movie_genre)/frame))
max_frames

In [ ]:
for i in np.arange(0, max_frames):
    df_aux = df_movie_genre.iloc[i*frame : (i+1)*frame ].copy()
    df_aux.apply(get_asins, axis=1, df_movie_meta=df_movie_meta)
    df_movie_meta.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/Temporal/movie_meta_valid_genres.csv')
    print('Iteration: ' + str(i))

In [ ]:
df_movie_meta.head()

In [ ]:
if len(df_movie_genre)%frame != 0:
    df_aux = df_movie_genre.iloc[max_frames*frame:].copy()
    df_aux.apply(get_asins, axis=1, df_movie_meta=df_movie_meta)

df_movie_meta.head()

In [ ]:
df_movie_meta.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_meta_valid_genres.csv', index=False)

In [ ]:
df_movie_meta[df_movie_meta['title clean'].str.contains(r'^toy story$')]['asin'].drop_duplicates()

In [ ]:
df_movie_meta.iloc[69898]

In [ ]:
df_movie_genre[df_movie_genre['title clean'] == 'jackie brown']

In [ ]:
df_movie_genre['title clean'][99]

In [ ]:
df_movie_genre['asins'] = df_movie_genre.apply(get_asins, axis=1, df_movie_meta=df_movie_meta)
df_movie_genre.head()

In [ ]:
asins = df_movie_meta[df_movie_meta['title clean'].str.contains('jackie brown')]['asin'].drop_duplicates()
asins

In [ ]:
df_movie_meta['asin'][0] in asins.values

In [ ]:
df_movie_meta['genres'] = df_movie_meta.apply(lambda x : df_movie_genre['genres'][99] if x['asin'] in asins.values else '', axis=1) 
df_movie_meta.head()

## Normalize data cleaned

### Book normalize

In [ ]:
df_bx_book = pd.read_csv('/home/ignacio/Datasets/Graph analysis/bx-valid-book.csv')
df_bx_book.head()

In [ ]:
df_book_valid = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/book_meta_valid_shelves.csv')
df_book_valid.head()

In [ ]:
df_book_valid = df_book_valid.dropna(subset=['ISBN'])
df_book_valid.head()

In [ ]:
df_book_valid['common-shelves'] = df_book_valid['common-shelves'].fillna('no-description')

In [ ]:
df_book_valid = df_book_valid[['ISBN', 'Book-Title', 'Book-Author', 'common-shelves', 'description', 'asin']]
df_book_valid.head()

In [ ]:
df_book_valid.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/book_meta_valid_shelves.csv', index=False)

### Movie normalize

In [ ]:
df_ml_movies = pd.read_csv('/home/ignacio/Datasets/Graph analysis/ml-valid-movies.csv')
df_ml_movies.head()

In [ ]:
df_ml_movies['title clean'] = get_title_clean(df_ml_movies, 'title')
df_ml_movies.head()

In [ ]:
df_movie_valid = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_meta_valid_genres.csv')
df_movie_valid.head()

In [ ]:
df_movie_valid['movieId'] = df_movie_valid['title clean'].apply(
    lambda x : df_ml_movies[df_ml_movies['title clean'].str.contains(x)]['movieId'].drop_duplicates().values )
df_movie_valid.head()

In [ ]:
df_movie_valid['movieId'] = df_movie_valid['movieId'].apply(lambda x : x[0] if len(x) != 0 else '')
df_movie_valid.head()

In [ ]:
df_movie_valid['director'] = df_movie_valid['director'].str.split(r', ')
df_movie_valid.head()

In [ ]:
df_movie_valid['director'] = df_movie_valid['director'].apply(lambda x : list(set(x)))
df_movie_valid.head()

In [ ]:
df_movie_valid['director'] = df_movie_valid['director'].apply(lambda x : ', '.join(x) if len(x) > 1 else x[0])
df_movie_valid.head()

In [ ]:
df_movie_valid = df_movie_valid[['movieId', 'title', 'genres', 'director', 'description', 'asin']]
df_movie_valid.head()

In [ ]:
df_movie_valid.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_meta_valid_genres.csv', index=False)

### movie-rating normalize

In [ ]:
df_movie_ratings = pd.read_csv('/home/ignacio/Datasets/Graph analysis/ml-ratings.csv')
df_movie_ratings.head()

In [2]:
df_movie_ratings = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_movie_intersect_ii.csv')
df_movie_ratings.head()

,userId,asin,rating,reviewTime,movieId
0,AGEIT17HENDIS,0764003828,3.0,1185062400,1963
1,A1GGOC9PVDXW7Z,0764003828,2.0,1280016000,1963
2,A34PAZQ73SL163,0764003828,5.0,1110585600,1963
3,A20IIR0422G3A5,0764003828,5.0,1115078400,1963
4,AW3VZ5O895LRK,0764003828,4.0,1379116800,1963


In [ ]:
len(df_movie_ratings)

In [ ]:
df_join_movie_ratings = pd.merge(df_movie_ratings, df_movie_valid[['asin', 'movieId']], how='left', left_on='asin', right_on='asin')
df_join_movie_ratings.head()

In [ ]:
len(df_join_movie_ratings)

In [ ]:
df_join_movie_ratings = df_join_movie_ratings.rename(index=str, columns={'reviewerID': 'userId'})
df_join_movie_ratings.head()

In [ ]:
df_join_movie_ratings.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_movie_intersect_ii.csv', index=False)

### book-rating normalize

In [5]:
df_book_valid = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/book_meta_valid_shelves.csv')
df_book_valid.head()

,ISBN,Book-Title,Book-Author,common-shelves,description,asin
0,0399135782,The Kitchen God's Wife,Amy Tan,romance|war|drama,Tan can relax. If The Joy Luck Club was an ast...,0143038109
1,0446310786,To Kill a Mockingbird,Harper Lee,crime|mystery|drama,"""When he was nearly thirteen, my brother Jem g...",0060194995
2,055321215X,Pride and Prejudice,Jane Austen,historical-romance|drama,no description,0030514878
3,0553582747,From the Corner of His Eye,Dean Koontz,fantasy|urban-fantasy|horror-suspense|suspense...,Horrormeister Koontz looks heavenward for insp...,0553502697
4,0380715899,A Soldier of the Great War,Mark Helprin,the-great-war|adventure|war-memoirs|world-war-...,"With energetic, often lyrical prose capable of...",0151836000


In [3]:
df_book_ratings_join = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_book_intersect_ii.csv')
df_book_ratings_join.head()

,reviewerID,asin,rating,reviewTime
0,AGEIT17HENDIS,0002051850,4.0,1148860800
1,A2RM9L2MM9EQVE,0002051850,5.0,1222905600
2,AHCVWPLA1O4X8,0002051850,5.0,1059696000
3,A4BNZ1OSBO77S,0002051850,5.0,1059350400
4,A3EYOITBD7ZXOV,0002051850,2.0,1198627200


In [4]:
df_book_ratings_join = df_book_ratings_join.rename(index=str, columns={'reviewerID': 'userId'})
df_book_ratings_join.head()

,userId,asin,rating,reviewTime
0,AGEIT17HENDIS,0002051850,4.0,1148860800
1,A2RM9L2MM9EQVE,0002051850,5.0,1222905600
2,AHCVWPLA1O4X8,0002051850,5.0,1059696000
3,A4BNZ1OSBO77S,0002051850,5.0,1059350400
4,A3EYOITBD7ZXOV,0002051850,2.0,1198627200


In [6]:
df_book_ratings_join = pd.merge(df_book_ratings_join, df_book_valid[['asin', 'ISBN']], how='left', right_on='asin', left_on='asin')
df_book_ratings_join.head()

,userId,asin,rating,reviewTime,ISBN
0,AGEIT17HENDIS,0002051850,4.0,1148860800,0684803356
1,A2RM9L2MM9EQVE,0002051850,5.0,1222905600,0684803356
2,AHCVWPLA1O4X8,0002051850,5.0,1059696000,0684803356
3,A4BNZ1OSBO77S,0002051850,5.0,1059350400,0684803356
4,A3EYOITBD7ZXOV,0002051850,2.0,1198627200,0684803356


In [9]:
len(df_book_ratings_join)

29695

In [10]:
df_book_ratings_join = df_book_ratings_join.dropna(subset=['ISBN'])
len(df_book_ratings_join)

29694

In [11]:
df_book_ratings_join.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_book_intersect_ii.csv', index=False)

In [ ]:
df_movie_valid['genres'] = df_movie_valid['genres'].apply(lambda x : x[1:] if x[0] == '|' else x)
df_movie_valid['director'] = df_movie_valid['director'].apply(lambda x : x[2:] if x[:2] == ', ' else x)
df_movie_valid.head()

In [ ]:
len(df_movie_valid)

In [ ]:
df_movie_valid['director'][0]

In [ ]:
df_movie_valid.isnull().any()